#[رضا ناصر-علاء مرعي - علا ابو كرم]

In [1]:
!pip install camel-tools --upgrade
!pip install emoji
!pip install arabert
!camel_data -i light
!pip install transformers
!pip install --upgrade camel_tools emoji

  Using cached emoji-1.4.2-py3-none-any.whl
  Attempting uninstall: emoji
    Found existing installation: emoji 2.9.0
    Uninstalling emoji-2.9.0:
      Successfully uninstalled emoji-2.9.0
No new packages will be installed.
  Using cached emoji-2.9.0-py2.py3-none-any.whl (397 kB)
  Attempting uninstall: emoji
    Found existing installation: emoji 1.4.2
    Uninstalling emoji-1.4.2:
      Successfully uninstalled emoji-1.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arabert 1.0.1 requires emoji==1.4.2, but you have emoji 2.9.0 which is incompatible.


In [2]:
!pip install sklearn-crfsuite


In [3]:
import emoji
# using the thumbs up CLDR short name
print(emoji.emojize('Python is really cool :thumbs_up:'))
# using the grinning face CLDR short name
print(emoji.emojize('Python is really cool :grinning_face:'))

Python is really cool 👍
Python is really cool 😀


In [4]:
import tokenization

h1 style="text-align: center;">Sequence Labeling<h1>

Prepare libraries

In [5]:
# here put every import you need e.g. import nltk
# it's better to load what you need from the package by from [] import [] instead of import the whole package
# your code here
import nltk
from nltk.tokenize import RegexpTokenizer, TreebankWordTokenizer, TweetTokenizer, TreebankWordDetokenizer
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt            # library for visualization
import matplotlib
import matplotlib.patches as mpatches
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import accuracy_score
from wordcloud import WordCloud

Download data

In [6]:
!wget 'https://drive.google.com/uc?export=download&id=1-3JRPU4aq9t_RRujrJ1HpmAPUbRe0QYK' -O 'IOB.csv'

--2023-12-30 20:43:23--  https://drive.google.com/uc?export=download&id=1-3JRPU4aq9t_RRujrJ1HpmAPUbRe0QYK
Resolving drive.google.com (drive.google.com)... 209.85.200.138, 209.85.200.101, 209.85.200.113, ...
Connecting to drive.google.com (drive.google.com)|209.85.200.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-00-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ieult19cil20cnv7ugnp4e1p6ech0iu7/1703968950000/04260309330816471542/*/1-3JRPU4aq9t_RRujrJ1HpmAPUbRe0QYK?e=download&uuid=da425e35-431a-4612-a044-a5f7e28bf022 [following]
--2023-12-30 20:43:24--  https://doc-0k-00-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ieult19cil20cnv7ugnp4e1p6ech0iu7/1703968950000/04260309330816471542/*/1-3JRPU4aq9t_RRujrJ1HpmAPUbRe0QYK?e=download&uuid=da425e35-431a-4612-a044-a5f7e28bf022
Resolving doc-0k-00-docs.googleusercontent.com (doc-0k-00-docs.googleusercontent.com)... 172.217.212.132, 2607

<h2 dir="rtl">مثال عن كيفية كتابة حلول الطلبات:</h2>


<div dir="rtl">شرح ما يقوم به الكود (like code documentation)<div>

In [7]:
# your code here
Data_df = pd.read_csv('IOB.csv')
len(Data_df)

62506

In [8]:
# example te(st
Data_df

,Word i,Word i entity tag,Word i POS,Stopword
0,إن,O,verb_pseudo,yes
1,اللوكيميا,B,noun,no
2,أو,O,conj,yes
3,(,O,punc,no
4,ابيضاض,B,noun,no
...,...,...,...,...
62501,ب,O,prep,no
62502,سرطان,B,noun,no
62503,عنق,I,noun,no
62504,الرحم,I,noun,no


<div dir="rtl">ملاحظاتك في حال وجودها</div>
<div dir="rtl">يمكنك إضافة خلايا لكل طلب بقدر ما تشاء، المهم أن تحافظ على تنظيم الملف</div>

# Question [1]: Data Preparation

## [1.1]

In [9]:
def lines_to_tuples(data_df):
    # قائمة لتخزين tuples
    tuples = []

    # حلقة لتحويل كل سطر إلى tuple
    for i in range(len(data_df)):
        # استخراج الكلمة وتصنيفها المتوقع entity tag
        word = data_df.loc[i, 'Word i']
        entity_tag = data_df.loc[i, 'Word i entity tag']
        # إضافة tuple جديد إلى قائمة الtuples
        tuples.append((word, entity_tag))

    return tuples



## [1.2]

In [10]:

def lines_to_sentences(data_df):
    # قائمة لتخزين الجمل
    sentences = []
    # قائمة مؤقتة لتخزين مفردات الجملة
    sentence_words = []

    # حلقة لتحويل الأسطر إلى جمل
    for i in range(len(data_df)):
        # استخراج الكلمة
        word = data_df.loc[i, 'Word i']
        # إضافة الكلمة إلى قائمة مفردات الجملة
        sentence_words.append(word)

        # إذا واجهنا نقطة في الأسطر، نعتبر أنها نهاية الجملة
        if word == '.':
            # إضافة قائمة مفردات الجملة إلى قائمة الجمل
            sentences.append(sentence_words)
            # إعادة قائمة مفردات الجملة إلى الحالة الأولية للجملة الجديدة
            sentence_words = []

    return sentences
lines_to_sentences = lines_to_sentences(Data_df)
print(lines_to_sentences)

[['إن', 'اللوكيميا', 'أو', '(', 'ابيضاض', 'الدم', ')', 'هو', 'سرطان', 'خلايا', 'الدم', 'البيضاء', ',', 'و', 'خلايا', 'الدم', 'البيضاء', 'تساعد', 'الجسم', 'على', 'محاربة', 'العدوى', '.'], ['تتشكل', 'خلايا', 'الدم', 'في', 'نقي', 'العظام', ',', 'أما', 'في', 'اللوكيميا', 'ف', 'يقوم', 'نقي', 'العظام', 'على', 'كل', 'حال', 'ب', 'إنتاج', 'خلايا', 'دم', 'بيضاء', 'غير', 'طبيعية', ',', 'ب', 'حيث', 'تتجمع', 'تلك', 'الخلايا', 'حول', 'خلايا', 'الدم', 'الصحيحة', 'مما', 'يجعل', 'من', 'الصعب', 'أن', 'تقوم', 'هذه', 'الخلايا', 'ب', 'عملها', 'ب', 'شكل', 'عادي', '.'], ['هناك', 'أنواع', 'مختلفة', 'من', 'اللوكيميا', ',', 'و', 'تتضمن', 'هذه', 'الأنواع', ':', 'ابيضاض', 'اللمفاويات', 'الحاد', 'الابيضاض', 'النخاعي', 'الحاد', 'ابيضاض', 'اللمفاويات', 'المزمن', 'الابيضاض', 'النخاعي', 'المزمن', 'يمكن', 'ل', 'اللوكيميا', 'أن', 'تتطور', 'ب', 'سرعة', 'أو', 'ب', 'بطء', ',', 'ف', 'اللوكيميا', 'المزمنة', 'تتطور', 'ب', 'بطء', ',', 'أما', 'في', 'اللوكيميا', 'الحادة', 'فتكون', 'الخلايا', 'شاذة', 'جدا', 'عن', 'الوضع', 'الطبيع

## [1.3]

In [11]:
def lines_to_lists(data_df):
    # قائمة لتخزين الجمل
    sentences = []
    # قائمة مؤقتة لتخزين tuples المفردات والتصنيفات
    sentence_tokens = []

    # حلقة لتحويل الأسطر إلى قوائم
    for i in range(len(data_df)):
        # استخراج الكلمة والتصنيف المتوقع
        word = data_df.loc[i, 'Word i']
        tag = data_df.loc[i, 'Word i entity tag']
        # إضافة tuple من الكلمة والتصنيف إلى قائمة المفردات والتصنيفات
        sentence_tokens.append((word, tag))

        # إذا واجهنا نقطة في الأسطر، نعتبر أنها نهاية الجملة
        if word == '.':
            # إضافة قائمة المفردات والتصنيفات إلى قائمة الجمل
            sentences.append(sentence_tokens)
            # إعادة قائمة المفردات والتصنيفات إلى الحالة الأولية للجملة الجديدة
            sentence_tokens = []

    return sentences


## [1.4]

In [12]:
def sentences_POS_Lists(data_df):
    sentences_lists = []
    pos_lists=[]
    sentence_words = []
    pos_words=[]
    for i in range(len(data_df)):
        word = data_df.loc[i, 'Word i']
        pos = data_df.loc[i, 'Word i POS']
        sentence_words.append(word)
        pos_words.append(pos)
        if word == '.'and pos =='punc':
            sentences_lists.append(sentence_words)
            pos_lists.append(pos_words)
            sentence_words = []
            pos_words=[]

    return sentences_lists,pos_lists

sentences_lists, POS_lists = sentences_POS_Lists(Data_df)


print("اخر عنصر من الجملة")
print(sentences_lists[-1][-1])

print("POS اخر عنصر من الجملة")
print(POS_lists[-1][-1])

اخر عنصر من الجملة
.
POS اخر عنصر من الجملة
punc


## [1.5]

In [13]:
from sklearn.model_selection import train_test_split

def extract_lists(data_df):
    word_lists = []
    entity_lists = []

    for index, row in data_df.iterrows():
        word = row['Word i']
        entity = row['Word i entity tag']

        word_lists.append(word)
        entity_lists.append(entity)

    return word_lists, entity_lists

# قراءة ملف CSV
data_df = pd.read_csv('IOB.csv')

# تقسيم مجموعة البيانات إلى مجموعة تدريب واختبار
train_df, test_df = train_test_split(data_df, test_size=0.2, shuffle=False)

train_df=train_df.reset_index()

test_df=test_df.reset_index()

# استخراج القوائم لمجموعة التدريب
train_word_list, train_entity_list = extract_lists(train_df)

# استخراج القوائم لمجموعة الاختبار
test_word_list, test_entity_list = extract_lists(test_df)

# طباعة قوائم مجموعة التدريب
print("قائمة مفردات مجموعة التدريب:")
print(train_word_list[5])

print("قائمة تصنيف كلمات مجموعة التدريب:")
print(train_entity_list[5])

# طباعة قوائم مجموعة الاختبار
print("قائمة مفردات مجموعة الاختبار:")
print(test_word_list[5])

print("قائمة تصنيف كلمات مجموعة الاختبار:")
print(test_entity_list[5])

قائمة مفردات مجموعة التدريب:
الدم
قائمة تصنيف كلمات مجموعة التدريب:
I
قائمة مفردات مجموعة الاختبار:
واحدة
قائمة تصنيف كلمات مجموعة الاختبار:
O


## [1.6]

In [14]:
# حساب عدد الجمل في مجموعة التدريب
train_sentence_count = len(train_word_list)

# حساب عدد الجمل في مجموعة الاختبار
test_sentence_count = len(test_word_list)
print("عدد الجمل في مجموعة التدريب:", train_sentence_count)
print("عدد الجمل في مجموعة الاختبار:", test_sentence_count)

عدد الجمل في مجموعة التدريب: 50004
عدد الجمل في مجموعة الاختبار: 12502


## [1.7]

In [15]:
def count_entities(entity_list):
    entity_count = 0

    for entity in entity_list:
        if entity != 'O':
            entity_count += 1

    return entity_count

def calculate_entity_ratio(entity_count, word_count):
    ratio = entity_count / word_count * 100
    return ratio

# حساب عدد الكيانات الاسمية في مجموعة التدريب
train_entity_count = count_entities(train_entity_list)

# حساب عدد الكلمات الكلي في مجموعة التدريب
train_word_count = len(train_word_list)

# حساب نسبة الكيانات الاسمية إلى عدد الكلمات الكلي في مجموعة التدريب
train_entity_ratio = calculate_entity_ratio(train_entity_count, train_word_count)

# طباعة عدد الكيانات الاسمية ونسبتها في مجموعة التدريب
print("عدد الكيانات الاسمية في مجموعة التدريب:", train_entity_count)
print("نسبة الكيانات الاسمية إلى عدد الكلمات الكلي في مجموعة التدريب:", train_entity_ratio, "%")

عدد الكيانات الاسمية في مجموعة التدريب: 2300
نسبة الكيانات الاسمية إلى عدد الكلمات الكلي في مجموعة التدريب: 4.599632029437645 %


## [1.8]

In [16]:
def count_ambiguous_words(word_list):
    ambiguous_word_count = 0

    for word in word_list:
        if word == 'O':
            ambiguous_word_count += 1

    return ambiguous_word_count

def calculate_ambiguous_word_ratio(ambiguous_word_count, word_count):
    ratio = ambiguous_word_count / word_count * 100
    return ratio

# حساب عدد الكلمات الغامضة في مجموعة التدريب
train_ambiguous_word_count = count_ambiguous_words(train_entity_list)

# حساب عدد الكلمات الكلي في مجموعة التدريب
train_word_count = len(train_word_list)

# حساب نسبة الكلمات الغامضة إلى عدد الكلمات الكلي في مجموعة التدريب
train_ambiguous_word_ratio = calculate_ambiguous_word_ratio(train_ambiguous_word_count, train_word_count)

# طباعة عدد الكلمات الغامضة ونسبتها في مجموعة التدريب
print("عدد الكلمات الغامضة في مجموعة التدريب:", train_ambiguous_word_count)
print("نسبة الكلمات الغامضة إلى عدد الكلمات الكلي في مجموعة التدريب:", train_ambiguous_word_ratio, "%")

عدد الكلمات الغامضة في مجموعة التدريب: 47704
نسبة الكلمات الغامضة إلى عدد الكلمات الكلي في مجموعة التدريب: 95.40036797056234 %



[1.9]

In [17]:
most_common_label = train_df['Word i entity tag'].value_counts()

print("The most frequent tag is:", most_common_label)

The most frequent tag is: O    47704
I     1254
B     1046
Name: Word i entity tag, dtype: int64


# Question [2]: Statistics

In [18]:
from tokenization import tokenize


In [19]:
raw_sent = "التصلب المتعدد مرض يحتمل أن يسبب إعاقة في الدماغ والحبل الشوكي (الجهاز العصبي المركزي). عند الإصابة بمرض التصلب المتعدد، يهاجم الجهاز المناعي غمد الحماية (المايلين) الذي يغطي الألياف العصبية، ويسبب مشكلات في الاتصال بين الدماغ وبقية الجسم. في النهاية، يمكن أن يسبب المرض تلفًا أو تدهورًا دائمًا في الألياف العصبية وقد يؤدي الى امراض الدم."
sent_tokens = tokenization.tokenize(raw_sent)

## [2.1]


In [20]:
default_tagger = nltk.DefaultTagger('O')
tagged=default_tagger.tag(train_df['Word i'])
print(tagged)

[('إن', 'O'), ('اللوكيميا', 'O'), ('أو', 'O'), ('(', 'O'), ('ابيضاض', 'O'), ('الدم', 'O'), (')', 'O'), ('هو', 'O'), ('سرطان', 'O'), ('خلايا', 'O'), ('الدم', 'O'), ('البيضاء', 'O'), (',', 'O'), ('و', 'O'), ('خلايا', 'O'), ('الدم', 'O'), ('البيضاء', 'O'), ('تساعد', 'O'), ('الجسم', 'O'), ('على', 'O'), ('محاربة', 'O'), ('العدوى', 'O'), ('.', 'O'), ('تتشكل', 'O'), ('خلايا', 'O'), ('الدم', 'O'), ('في', 'O'), ('نقي', 'O'), ('العظام', 'O'), (',', 'O'), ('أما', 'O'), ('في', 'O'), ('اللوكيميا', 'O'), ('ف', 'O'), ('يقوم', 'O'), ('نقي', 'O'), ('العظام', 'O'), ('على', 'O'), ('كل', 'O'), ('حال', 'O'), ('ب', 'O'), ('إنتاج', 'O'), ('خلايا', 'O'), ('دم', 'O'), ('بيضاء', 'O'), ('غير', 'O'), ('طبيعية', 'O'), (',', 'O'), ('ب', 'O'), ('حيث', 'O'), ('تتجمع', 'O'), ('تلك', 'O'), ('الخلايا', 'O'), ('حول', 'O'), ('خلايا', 'O'), ('الدم', 'O'), ('الصحيحة', 'O'), ('مما', 'O'), ('يجعل', 'O'), ('من', 'O'), ('الصعب', 'O'), ('أن', 'O'), ('تقوم', 'O'), ('هذه', 'O'), ('الخلايا', 'O'), ('ب', 'O'), ('عملها', 'O'), ('ب', 

In [21]:
tagged_test_sentences = lines_to_lists(test_df)


In [22]:
default_tagger.evaluate(tagged_test_sentences)

<ipython-input-22-1688dfa7520c>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  default_tagger.evaluate(tagged_test_sentences)


0.9574468085106383

In [23]:
print(default_tagger.tag(sent_tokens))

[('التصلب', 'O'), ('المتعدد', 'O'), ('مرض', 'O'), ('يحتمل', 'O'), ('أن', 'O'), ('يسبب', 'O'), ('أعاقه', 'O'), ('في', 'O'), ('الدماغ', 'O'), ('والحبل', 'O'), ('الشوكي', 'O'), ('(', 'O'), ('الجهاز', 'O'), ('العصبي', 'O'), ('المركزي', 'O'), (')', 'O'), ('.', 'O'), ('عند', 'O'), ('الإصابة', 'O'), ('ب', 'O'), ('مرض', 'O'), ('التصلب', 'O'), ('المتعدد', 'O'), ('،', 'O'), ('يهاجم', 'O'), ('الجهاز', 'O'), ('المناعي', 'O'), ('غمد', 'O'), ('الحماية', 'O'), ('(', 'O'), ('المايلين', 'O'), (')', 'O'), ('الذي', 'O'), ('يغطي', 'O'), ('الألياف', 'O'), ('العصبية', 'O'), ('،', 'O'), ('ويسبب', 'O'), ('مشكلات', 'O'), ('في', 'O'), ('الاتصال', 'O'), ('بين', 'O'), ('الدماغ', 'O'), ('وبقية', 'O'), ('الجسم', 'O'), ('.', 'O'), ('في', 'O'), ('النهاية', 'O'), ('،', 'O'), ('يمكن', 'O'), ('أن', 'O'), ('يسبب', 'O'), ('المرض', 'O'), ('تلفا', 'O'), ('أو', 'O'), ('تدهورا', 'O'), ('دائما', 'O'), ('في', 'O'), ('الألياف', 'O'), ('العصبية', 'O'), ('وقد', 'O'), ('يؤدي', 'O'), ('إلى', 'O'), ('أمراض', 'O'), ('الدم', 'O'), ('.'

## [2.2]

In [24]:
cfd = nltk.ConditionalFreqDist(lines_to_tuples(train_df))

In [25]:
fd = nltk.FreqDist(train_df['Word i'])

In [26]:
most_freq_words = fd.most_common(100)
print(most_freq_words)

[('.', 3038), ('و', 2291), ('من', 1569), (',', 1431), ('ب', 1288), ('في', 1059), ('ل', 903), ('سرطان', 821), ('أو', 652), ('على', 620), ('السرطان', 595), ('إلى', 564), ('أن', 509), ('يمكن', 390), ('المعالجة', 378), ('الجسم', 360), ('قد', 307), ('ف', 296), ('الخلايا', 286), ('هذه', 277), ('إن', 274), ('"', 272), ('عن', 265), ('إذا', 265), ('هذا', 263), ('ما', 247), ('الورم', 246), ('هو', 245), ('التي', 237), ('أيضا', 209), ('هي', 201), ('الطبيب', 195), ('لا', 191), ('الدم', 183), ('المريض', 183), ('الإصابة', 174), ('أجل', 173), ('كان', 165), ('غير', 161), (':', 150), ('الذي', 149), ('ذلك', 143), ('أخرى', 142), ('بعض', 138), ('الأعراض', 135), ('الفم', 134), ('خلايا', 131), ('اسم', 130), ('يجري', 129), ('تحديد', 123), ('يكون', 122), ('الجراحة', 120), ('كما', 119), ('أكثر', 119), ('هناك', 117), ('الكبد', 116), ('السرطانية', 114), ('العين', 111), ('معالجة', 110), ('إجراء', 109), ('الجلد', 105), ('عادة', 104), ('الكيميائية', 103), ('الثدي', 103), ('يدعى', 100), ('الممكن', 100), ('الرحم', 98)

In [27]:
likely_tags = dict((word, cfd[word].max()) for (word, _) in most_freq_words)
print(likely_tags)
baseline_tagger = nltk.UnigramTagger(model=likely_tags)



{'.': 'O', 'و': 'O', 'من': 'O', ',': 'O', 'ب': 'O', 'في': 'O', 'ل': 'O', 'سرطان': 'B', 'أو': 'O', 'على': 'O', 'السرطان': 'O', 'إلى': 'O', 'أن': 'O', 'يمكن': 'O', 'المعالجة': 'O', 'الجسم': 'O', 'قد': 'O', 'ف': 'O', 'الخلايا': 'O', 'هذه': 'O', 'إن': 'O', '"': 'O', 'عن': 'O', 'إذا': 'O', 'هذا': 'O', 'ما': 'O', 'الورم': 'O', 'هو': 'O', 'التي': 'O', 'أيضا': 'O', 'هي': 'O', 'الطبيب': 'O', 'لا': 'O', 'الدم': 'O', 'المريض': 'O', 'الإصابة': 'O', 'أجل': 'O', 'كان': 'O', 'غير': 'O', ':': 'O', 'الذي': 'O', 'ذلك': 'O', 'أخرى': 'O', 'بعض': 'O', 'الأعراض': 'O', 'الفم': 'O', 'خلايا': 'O', 'اسم': 'O', 'يجري': 'O', 'تحديد': 'O', 'يكون': 'O', 'الجراحة': 'O', 'كما': 'O', 'أكثر': 'O', 'هناك': 'O', 'الكبد': 'O', 'السرطانية': 'O', 'العين': 'O', 'معالجة': 'O', 'إجراء': 'O', 'الجلد': 'O', 'عادة': 'O', 'الكيميائية': 'O', 'الثدي': 'O', 'يدعى': 'O', 'الممكن': 'O', 'الرحم': 'I', 'عبر': 'O', 'الأشعة': 'O', 'أجزاء': 'O', 'انتشر': 'O', 'بعد': 'O', 'تكون': 'O', 'العلاج': 'O', 'الدماغ': 'O', 'الأمعاء': 'O', 'الشرج': 'O

In [28]:
baseline_tagger.evaluate(tagged_test_sentences)

<ipython-input-28-8bb38bcc5481>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  baseline_tagger.evaluate(tagged_test_sentences)


0.5223964165733482

In [29]:
print(baseline_tagger.tag(sent_tokens))


[('التصلب', None), ('المتعدد', None), ('مرض', None), ('يحتمل', None), ('أن', 'O'), ('يسبب', None), ('أعاقه', None), ('في', 'O'), ('الدماغ', 'O'), ('والحبل', None), ('الشوكي', None), ('(', 'O'), ('الجهاز', None), ('العصبي', None), ('المركزي', None), (')', None), ('.', 'O'), ('عند', 'O'), ('الإصابة', 'O'), ('ب', 'O'), ('مرض', None), ('التصلب', None), ('المتعدد', None), ('،', None), ('يهاجم', None), ('الجهاز', None), ('المناعي', None), ('غمد', None), ('الحماية', None), ('(', 'O'), ('المايلين', None), (')', None), ('الذي', 'O'), ('يغطي', None), ('الألياف', None), ('العصبية', None), ('،', None), ('ويسبب', None), ('مشكلات', None), ('في', 'O'), ('الاتصال', None), ('بين', None), ('الدماغ', 'O'), ('وبقية', None), ('الجسم', 'O'), ('.', 'O'), ('في', 'O'), ('النهاية', None), ('،', None), ('يمكن', 'O'), ('أن', 'O'), ('يسبب', None), ('المرض', None), ('تلفا', None), ('أو', 'O'), ('تدهورا', None), ('دائما', None), ('في', 'O'), ('الألياف', None), ('العصبية', None), ('وقد', None), ('يؤدي', None), ('إلى'

Many words have been assigned a tag of None, because they were not among the 100 most frequent words.


**Backoff tagging**

In [30]:
backoff_tagger = nltk.UnigramTagger(model=likely_tags,backoff=nltk.DefaultTagger('O'))

In [31]:
backoff_tagger.evaluate(tagged_test_sentences)

<ipython-input-31-01d869d0eeac>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  backoff_tagger.evaluate(tagged_test_sentences)


0.97216445368741

## [2.3]

**Unigram Tagging**

In [32]:
tagged_train_sentences=lines_to_lists(train_df)
print(tagged_train_sentences)
unigram_tagger = nltk.UnigramTagger(tagged_train_sentences)

print(unigram_tagger.tag(sent_tokens))

[[('إن', 'O'), ('اللوكيميا', 'B'), ('أو', 'O'), ('(', 'O'), ('ابيضاض', 'B'), ('الدم', 'I'), (')', 'O'), ('هو', 'O'), ('سرطان', 'B'), ('خلايا', 'I'), ('الدم', 'I'), ('البيضاء', 'I'), (',', 'O'), ('و', 'O'), ('خلايا', 'O'), ('الدم', 'O'), ('البيضاء', 'O'), ('تساعد', 'O'), ('الجسم', 'O'), ('على', 'O'), ('محاربة', 'O'), ('العدوى', 'O'), ('.', 'O')], [('تتشكل', 'O'), ('خلايا', 'O'), ('الدم', 'O'), ('في', 'O'), ('نقي', 'O'), ('العظام', 'O'), (',', 'O'), ('أما', 'O'), ('في', 'O'), ('اللوكيميا', 'B'), ('ف', 'O'), ('يقوم', 'O'), ('نقي', 'O'), ('العظام', 'O'), ('على', 'O'), ('كل', 'O'), ('حال', 'O'), ('ب', 'O'), ('إنتاج', 'O'), ('خلايا', 'O'), ('دم', 'O'), ('بيضاء', 'O'), ('غير', 'O'), ('طبيعية', 'O'), (',', 'O'), ('ب', 'O'), ('حيث', 'O'), ('تتجمع', 'O'), ('تلك', 'O'), ('الخلايا', 'O'), ('حول', 'O'), ('خلايا', 'O'), ('الدم', 'O'), ('الصحيحة', 'O'), ('مما', 'O'), ('يجعل', 'O'), ('من', 'O'), ('الصعب', 'O'), ('أن', 'O'), ('تقوم', 'O'), ('هذه', 'O'), ('الخلايا', 'O'), ('ب', 'O'), ('عملها', 'O'), ('ب

In [33]:
unigram_tagger.evaluate(tagged_test_sentences)

<ipython-input-33-616291637d8b>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  unigram_tagger.evaluate(tagged_test_sentences)


0.9258518637018077

**Bigram Tagger**

In [34]:
bigram_tagger = nltk.BigramTagger(tagged_train_sentences)

In [35]:
bigram_tagger.evaluate(tagged_test_sentences)

<ipython-input-35-0437d767aa26>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  bigram_tagger.evaluate(tagged_test_sentences)


0.5759878419452887

In [36]:
print(bigram_tagger.tag(sent_tokens))

[('التصلب', None), ('المتعدد', None), ('مرض', None), ('يحتمل', None), ('أن', None), ('يسبب', None), ('أعاقه', None), ('في', None), ('الدماغ', None), ('والحبل', None), ('الشوكي', None), ('(', None), ('الجهاز', None), ('العصبي', None), ('المركزي', None), (')', None), ('.', None), ('عند', None), ('الإصابة', None), ('ب', None), ('مرض', None), ('التصلب', None), ('المتعدد', None), ('،', None), ('يهاجم', None), ('الجهاز', None), ('المناعي', None), ('غمد', None), ('الحماية', None), ('(', None), ('المايلين', None), (')', None), ('الذي', None), ('يغطي', None), ('الألياف', None), ('العصبية', None), ('،', None), ('ويسبب', None), ('مشكلات', None), ('في', None), ('الاتصال', None), ('بين', None), ('الدماغ', None), ('وبقية', None), ('الجسم', None), ('.', None), ('في', None), ('النهاية', None), ('،', None), ('يمكن', None), ('أن', None), ('يسبب', None), ('المرض', None), ('تلفا', None), ('أو', None), ('تدهورا', None), ('دائما', None), ('في', None), ('الألياف', None), ('العصبية', None), ('وقد', None), ('ي

# Question [3]: ML Classification

## [3.1]


In [37]:
#sentences_lists, POS_lists = sentences_POS_Lists(Data_df)


### [3.1.1]

In [38]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
    }

### [3.1.2]

In [39]:
def features_prev_next(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '<START>' if index == 0 else sentence[index - 1],
        'next_word': '<END>' if index == len(sentence) - 1 else sentence[index + 1],
    }


### [3.1.3]

In [40]:
def features_word_pos(sentence,pos, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'word_POS': pos[index]
    }


### [3.1.4]

In [41]:
def features_prev_next_pos(sentence, pos,index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word_pos': '<START>' if index == 0 else pos[index - 1],
        'next_word_pos': '<END>' if index == len(sentence) - 1 else pos[index + 1],
    }


### [3.1.5]

In [42]:
def lines_to_lists_pos(data_df):
    sentences = []
    sentence_tokens = []

    for i in range(len(data_df)):
        word = data_df.loc[i, 'Word i']
        tag = data_df.loc[i, 'Word i POS']
        sentence_tokens.append((word, tag))

        if word == '.':
            sentences.append(sentence_tokens)
            sentence_tokens = []

    return sentences
pos_tagged_sentences=lines_to_lists_pos(train_df)

In [43]:
bigram_tagger2 = nltk.BigramTagger(pos_tagged_sentences,backoff=nltk.DefaultTagger('noun'))

In [44]:
def features_bigram(sentence,index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    sentence_tags=bigram_tagger2.tag(sentence)
    _,x=sentence_tags[index]
    try:
      _,y=sentence_tags[index - 1]
    except:
      pass
    try:
      _,z=sentence_tags[index + 1]
    except:
      pass
    return {
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'word_bigram': x ,
        'prev_word_bigram': '<START>' if index == 0 else y,
        'next_word_bigram': '<END>' if index == len(sentence) - 1 else z,
    }


In [45]:
features_bigram(lines_to_sentences[66],2)

{'prefix-1': 'ا',
 'prefix-2': 'اب',
 'prefix-3': 'ابي',
 'suffix-1': 'ض',
 'suffix-2': 'اض',
 'suffix-3': 'ضاض',
 'word_bigram': 'noun',
 'prev_word_bigram': 'noun',
 'next_word_bigram': 'noun'}

### [3.1.6]

In [46]:
def consecutive_features(sentence, index, history):
    """ sentence: [w1, w2, ...],
    index: the index of the word """
    return {
        'word': sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '<START>' if index == 0 else sentence[index - 1],
        'prev_tag' : '<START>' if index == 0 else history[index - 1],
        'next_word': '<END>' if index == len(sentence) - 1 else sentence[index + 1],
    }


### [3.1.7]

## [3.2]

In [47]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
import datetime

In [48]:
pos_tagged_train_sentences=lines_to_lists_pos(train_df)
iob_tagged_train_sentences=lines_to_lists(train_df)


pos_tagged_test_sentences=lines_to_lists_pos(test_df)
iob_tagged_test_sentences=lines_to_lists(test_df)

In [49]:
comparison_table = {}

comparison_table['question_step_number'] = []
comparison_table['model_name'] = []
comparison_table['features'] = []
comparison_table['Training time'] = []
comparison_table['accuracy'] = []

In [50]:
def consecutive_transform_to_dataset(iob_tagged_sentences):
    X, y = [], []
    for tagged in iob_tagged_sentences:
        for index, word in enumerate(tagged):
            untagged = [w for w, _ in tagged]
            tags =     [t for _, t in tagged]
            X.append(consecutive_features(untagged, index, tags))
    # y should be in iob
            y.append(tags[index])
    return X, y

In [51]:


def consecutive_classify(sentence):
    history = []
    for i in range(len(sentence)):
        features = consecutive_features(sentence, i, history)
        X = vectorizer.transform(features)
        tag = classifier.predict(X)
        #print(tag)
        history.append(tag[0])
    return list(zip(sentence,history))

def consecutive_accurecy(test_sentences):
    count_all = 0.0
    count_true = 0.0
    for tagged_sentence in test_sentences:
        words_test = [w for w,_ in tagged_sentence]
        tags_true =  [t for _, t in tagged_sentence]
        predicted_sentence = consecutive_classify(words_test)
        predicted_tags = [t for _, t in predicted_sentence]
        for i, tag in enumerate(tags_true):
            if (predicted_tags[i] == tag):
                count_true+=1
            count_all +=1
    return count_true / count_all

In [52]:
def consecutive_train(X,y,classifier,vectorizer, cutoff_size=0):

    if (cutoff_size > 0):
        XX = vectorizer.fit_transform(X[:cutoff_size])
    else:
        XX = vectorizer.fit_transform(X)

    d1 = datetime.datetime.now()
    if (cutoff_size > 0):
        classifier.fit(XX[:cutoff_size], y[:cutoff_size])
    else:
        classifier.fit(XX, y)
    d2 = datetime.datetime.now()
    return str(d2-d1)

In [53]:
X, y = consecutive_transform_to_dataset(iob_tagged_train_sentences)

In [54]:
classifier = MultinomialNB()
vectorizer = DictVectorizer(sparse=False)

training_time=consecutive_train(X ,y ,classifier,vectorizer,5000)

In [55]:
print(consecutive_classify(sent_tokens))

[('التصلب', 'O'), ('المتعدد', 'O'), ('مرض', 'O'), ('يحتمل', 'O'), ('أن', 'O'), ('يسبب', 'O'), ('أعاقه', 'O'), ('في', 'O'), ('الدماغ', 'O'), ('والحبل', 'O'), ('الشوكي', 'O'), ('(', 'O'), ('الجهاز', 'O'), ('العصبي', 'O'), ('المركزي', 'O'), (')', 'O'), ('.', 'O'), ('عند', 'O'), ('الإصابة', 'O'), ('ب', 'O'), ('مرض', 'O'), ('التصلب', 'O'), ('المتعدد', 'O'), ('،', 'O'), ('يهاجم', 'O'), ('الجهاز', 'O'), ('المناعي', 'O'), ('غمد', 'O'), ('الحماية', 'O'), ('(', 'O'), ('المايلين', 'O'), (')', 'O'), ('الذي', 'O'), ('يغطي', 'O'), ('الألياف', 'O'), ('العصبية', 'O'), ('،', 'O'), ('ويسبب', 'O'), ('مشكلات', 'O'), ('في', 'O'), ('الاتصال', 'O'), ('بين', 'O'), ('الدماغ', 'O'), ('وبقية', 'O'), ('الجسم', 'O'), ('.', 'O'), ('في', 'O'), ('النهاية', 'O'), ('،', 'O'), ('يمكن', 'O'), ('أن', 'O'), ('يسبب', 'O'), ('المرض', 'O'), ('تلفا', 'O'), ('أو', 'O'), ('تدهورا', 'O'), ('دائما', 'O'), ('في', 'O'), ('الألياف', 'O'), ('العصبية', 'O'), ('وقد', 'O'), ('يؤدي', 'O'), ('إلى', 'O'), ('أمراض', 'O'), ('الدم', 'I'), ('.'

In [56]:
comparison_table['question_step_number'].append("3.2")
comparison_table['model_name'].append('Multinomial naive bayes')
comparison_table['features'].append('consecutive_features')
comparison_table['Training time'].append(training_time)
comparison_table['accuracy'].append(consecutive_accurecy(iob_tagged_test_sentences))


In [57]:
comparison_table

{'question_step_number': ['3.2'],
 'model_name': ['Multinomial naive bayes'],
 'features': ['consecutive_features'],
 'Training time': ['0:00:00.553024'],
 'accuracy': [0.9791233402655575]}

In [58]:
def transform_to_dataset(pos_tagged_sentences,iob_tagged_sentences,features):
    X, y = [], []
    for tagged in pos_tagged_sentences:
        for index in range(len(tagged)):
            untagged = [w for w, _ in tagged]
            if features==features_word_pos or features==features_prev_next_pos:
              pos = [p for _, p in tagged]
              X.append(features(untagged,pos, index))

            else: X.append(features(untagged, index))

    for tagged in iob_tagged_sentences:
        for index in range(len(tagged)):
            y.append(tagged[index][1])
    return X, y

In [59]:
def train(features,classifier,vectorizer,model_name,question_step_numbe,cutoff_size=5000):
    X, y = transform_to_dataset(pos_tagged_train_sentences,iob_tagged_train_sentences,features)

    if (cutoff_size > 0):
        XX = vectorizer.fit_transform(X[:cutoff_size])
    else:
        XX = vectorizer.fit_transform(X)

    d1 = datetime.datetime.now()
    if (cutoff_size > 0):
        classifier.fit(XX[:cutoff_size], y[:cutoff_size])
    else:
        classifier.fit(XX, y)
    d2 = datetime.datetime.now()

    Training_time=str(d2-d1)

    #evalueate


    X_test, y_test = transform_to_dataset(pos_tagged_test_sentences,iob_tagged_test_sentences,features)
    accuracy=classifier.score(vectorizer.transform(X_test), y_test)

    print( f'classify on the given sentence using {features.__name__} features : ' )
    print(classify(features,classifier,vectorizer))

    comparison_table['question_step_number'].append(question_step_numbe)
    comparison_table['model_name'].append(model_name)
    comparison_table['features'].append(f'{features.__name__}')
    comparison_table['Training time'].append(Training_time)
    comparison_table['accuracy'].append(accuracy)


In [60]:

unigram_tagger2 = nltk.UnigramTagger(pos_tagged_train_sentences,backoff=nltk.DefaultTagger('noun'))

tagged_sent_tokens = unigram_tagger2.tag(sent_tokens)
tags = [tag for _, tag in tagged_sent_tokens]

def classify(f,classifier,vectorizer):
    history = []
    for i in range(len(sent_tokens)):
        if f==features_word_pos or f==features_prev_next_pos:
          feature = f(sent_tokens,tags, i)
        else:
          feature = f(sent_tokens, i)
        X = vectorizer.transform(feature)
        tag = classifier.predict(X)
        #print(tag)
        history.append(tag[0])
    return list(zip(sent_tokens,history))


In [61]:
features_list=[features ,features_prev_next,features_bigram,features_prev_next_pos , features_word_pos]

In [62]:
for i in features_list:
  classifier = MultinomialNB()
  vectorizer = DictVectorizer(sparse=False)
  train(i,classifier,vectorizer,'Multinomial naive bayes','3.2',cutoff_size=5000)

classify on the given sentence using features features : 
[('التصلب', 'O'), ('المتعدد', 'O'), ('مرض', 'O'), ('يحتمل', 'O'), ('أن', 'O'), ('يسبب', 'O'), ('أعاقه', 'O'), ('في', 'O'), ('الدماغ', 'O'), ('والحبل', 'O'), ('الشوكي', 'O'), ('(', 'O'), ('الجهاز', 'O'), ('العصبي', 'O'), ('المركزي', 'O'), (')', 'O'), ('.', 'O'), ('عند', 'O'), ('الإصابة', 'O'), ('ب', 'O'), ('مرض', 'O'), ('التصلب', 'O'), ('المتعدد', 'O'), ('،', 'O'), ('يهاجم', 'O'), ('الجهاز', 'O'), ('المناعي', 'O'), ('غمد', 'O'), ('الحماية', 'O'), ('(', 'O'), ('المايلين', 'O'), (')', 'O'), ('الذي', 'O'), ('يغطي', 'O'), ('الألياف', 'O'), ('العصبية', 'O'), ('،', 'O'), ('ويسبب', 'O'), ('مشكلات', 'O'), ('في', 'O'), ('الاتصال', 'O'), ('بين', 'O'), ('الدماغ', 'O'), ('وبقية', 'O'), ('الجسم', 'O'), ('.', 'O'), ('في', 'O'), ('النهاية', 'O'), ('،', 'O'), ('يمكن', 'O'), ('أن', 'O'), ('يسبب', 'O'), ('المرض', 'O'), ('تلفا', 'O'), ('أو', 'O'), ('تدهورا', 'O'), ('دائما', 'O'), ('في', 'O'), ('الألياف', 'O'), ('العصبية', 'O'), ('وقد', 'O'), ('يؤدي

In [63]:
comparison_table

{'question_step_number': ['3.2', '3.2', '3.2', '3.2', '3.2', '3.2'],
 'model_name': ['Multinomial naive bayes',
  'Multinomial naive bayes',
  'Multinomial naive bayes',
  'Multinomial naive bayes',
  'Multinomial naive bayes',
  'Multinomial naive bayes'],
 'features': ['consecutive_features',
  'features',
  'features_prev_next',
  'features_bigram',
  'features_prev_next_pos',
  'features_word_pos'],
 'Training time': ['0:00:00.553024',
  '0:00:00.195037',
  '0:00:00.160412',
  '0:00:00.047619',
  '0:00:00.052280',
  '0:00:00.066995'],
 'accuracy': [0.9791233402655575,
  0.9588865781474964,
  0.9687250039993601,
  0.9611262198048313,
  0.9641657334826428,
  0.9569668852983523]}

## [3.3]

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [65]:
classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs')
vectorizer = DictVectorizer(sparse=False)

In [66]:
X, y = consecutive_transform_to_dataset(iob_tagged_train_sentences)
training_time=consecutive_train(X ,y,classifier,vectorizer,5000)

In [67]:
print(consecutive_classify(sent_tokens))

[('التصلب', 'O'), ('المتعدد', 'O'), ('مرض', 'O'), ('يحتمل', 'O'), ('أن', 'O'), ('يسبب', 'O'), ('أعاقه', 'O'), ('في', 'O'), ('الدماغ', 'O'), ('والحبل', 'O'), ('الشوكي', 'O'), ('(', 'O'), ('الجهاز', 'O'), ('العصبي', 'O'), ('المركزي', 'O'), (')', 'O'), ('.', 'O'), ('عند', 'O'), ('الإصابة', 'O'), ('ب', 'O'), ('مرض', 'O'), ('التصلب', 'O'), ('المتعدد', 'O'), ('،', 'O'), ('يهاجم', 'O'), ('الجهاز', 'O'), ('المناعي', 'O'), ('غمد', 'O'), ('الحماية', 'O'), ('(', 'O'), ('المايلين', 'O'), (')', 'O'), ('الذي', 'O'), ('يغطي', 'O'), ('الألياف', 'O'), ('العصبية', 'O'), ('،', 'O'), ('ويسبب', 'O'), ('مشكلات', 'O'), ('في', 'O'), ('الاتصال', 'O'), ('بين', 'O'), ('الدماغ', 'O'), ('وبقية', 'O'), ('الجسم', 'O'), ('.', 'O'), ('في', 'O'), ('النهاية', 'O'), ('،', 'O'), ('يمكن', 'O'), ('أن', 'O'), ('يسبب', 'O'), ('المرض', 'O'), ('تلفا', 'O'), ('أو', 'O'), ('تدهورا', 'O'), ('دائما', 'O'), ('في', 'O'), ('الألياف', 'O'), ('العصبية', 'O'), ('وقد', 'O'), ('يؤدي', 'O'), ('إلى', 'O'), ('أمراض', 'O'), ('الدم', 'O'), ('.'

In [68]:
comparison_table['question_step_number'].append("3.3")
comparison_table['model_name'].append('Multinomial Logistic Regression')
comparison_table['features'].append('consecutive_features')
comparison_table['Training time'].append(training_time)
comparison_table['accuracy'].append(consecutive_accurecy(tagged_test_sentences))

In [69]:
for i in features_list:
  classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs')
  vectorizer = DictVectorizer(sparse=False)
  train(i,classifier,vectorizer,'Multinomial Logistic Regression','3.3',cutoff_size=5000)

classify on the given sentence using features features : 
[('التصلب', 'O'), ('المتعدد', 'O'), ('مرض', 'O'), ('يحتمل', 'O'), ('أن', 'O'), ('يسبب', 'O'), ('أعاقه', 'O'), ('في', 'O'), ('الدماغ', 'O'), ('والحبل', 'O'), ('الشوكي', 'O'), ('(', 'O'), ('الجهاز', 'O'), ('العصبي', 'O'), ('المركزي', 'O'), (')', 'O'), ('.', 'O'), ('عند', 'O'), ('الإصابة', 'O'), ('ب', 'O'), ('مرض', 'O'), ('التصلب', 'O'), ('المتعدد', 'O'), ('،', 'O'), ('يهاجم', 'O'), ('الجهاز', 'O'), ('المناعي', 'O'), ('غمد', 'O'), ('الحماية', 'O'), ('(', 'O'), ('المايلين', 'O'), (')', 'O'), ('الذي', 'O'), ('يغطي', 'O'), ('الألياف', 'O'), ('العصبية', 'O'), ('،', 'O'), ('ويسبب', 'O'), ('مشكلات', 'O'), ('في', 'O'), ('الاتصال', 'O'), ('بين', 'O'), ('الدماغ', 'O'), ('وبقية', 'O'), ('الجسم', 'O'), ('.', 'O'), ('في', 'O'), ('النهاية', 'O'), ('،', 'O'), ('يمكن', 'O'), ('أن', 'O'), ('يسبب', 'O'), ('المرض', 'O'), ('تلفا', 'O'), ('أو', 'O'), ('تدهورا', 'O'), ('دائما', 'O'), ('في', 'O'), ('الألياف', 'O'), ('العصبية', 'O'), ('وقد', 'O'), ('يؤدي

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


classify on the given sentence using features_bigram features : 
[('التصلب', 'O'), ('المتعدد', 'O'), ('مرض', 'O'), ('يحتمل', 'O'), ('أن', 'O'), ('يسبب', 'O'), ('أعاقه', 'O'), ('في', 'O'), ('الدماغ', 'O'), ('والحبل', 'O'), ('الشوكي', 'O'), ('(', 'O'), ('الجهاز', 'O'), ('العصبي', 'O'), ('المركزي', 'O'), (')', 'O'), ('.', 'O'), ('عند', 'O'), ('الإصابة', 'O'), ('ب', 'O'), ('مرض', 'O'), ('التصلب', 'O'), ('المتعدد', 'O'), ('،', 'O'), ('يهاجم', 'O'), ('الجهاز', 'O'), ('المناعي', 'O'), ('غمد', 'O'), ('الحماية', 'O'), ('(', 'O'), ('المايلين', 'O'), (')', 'O'), ('الذي', 'O'), ('يغطي', 'O'), ('الألياف', 'O'), ('العصبية', 'O'), ('،', 'O'), ('ويسبب', 'O'), ('مشكلات', 'O'), ('في', 'O'), ('الاتصال', 'O'), ('بين', 'O'), ('الدماغ', 'O'), ('وبقية', 'O'), ('الجسم', 'O'), ('.', 'O'), ('في', 'O'), ('النهاية', 'O'), ('،', 'O'), ('يمكن', 'O'), ('أن', 'O'), ('يسبب', 'O'), ('المرض', 'O'), ('تلفا', 'O'), ('أو', 'O'), ('تدهورا', 'O'), ('دائما', 'O'), ('في', 'O'), ('الألياف', 'O'), ('العصبية', 'O'), ('وقد', 'O'),

## [3.4]

In [70]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

In [71]:
def crf_transform_to_dataset(pos_tagged_sentences,iob_tagged_sentences,features):
    X, y = [], []
    #
    if features==consecutive_features:
        for tagged in iob_tagged_sentences:
          untagged = [w for w, _ in tagged]
          tag=[t for _, t in tagged]
          sentence_features = [features(untagged, index,tag) for index in range(len(tagged))]
          X.append(sentence_features)

    #
    else:
      for tagged in pos_tagged_sentences:
        untagged = [w for w, _ in tagged]
        if (features==features_word_pos or features==features_prev_next_pos):
          pos = [p for _, p in tagged]
          sentence_features = [features(untagged,pos, index) for index in range(len(tagged))]
          X.append(sentence_features)
        else:
          sentence_features = [features(untagged, index) for index in range(len(tagged))]
          X.append(sentence_features)

    #
    for tagged in pos_tagged_sentences:
      sentence_labels = [tagged[index][1] for index in range(len(tagged))]
      y.append(sentence_labels)
    return X, y

In [72]:
def crf_train(features,classifier,vectorizer,model_name,question_step_numbe,cutoff_size=5000):
    X, y = crf_transform_to_dataset(pos_tagged_train_sentences,iob_tagged_train_sentences,features)

    d1 = datetime.datetime.now()
    if (cutoff_size > 0):
        classifier.fit(X[:cutoff_size], y[:cutoff_size])
    else:
        classifier.fit(X, y)
    d2 = datetime.datetime.now()

    Training_time=str(d2-d1)

    #evalueate

    X_test, y_test = crf_transform_to_dataset(pos_tagged_test_sentences,iob_tagged_test_sentences,features)
    y_pred = classifier.predict(X_test)
    accuracy=metrics.flat_accuracy_score(y_test, y_pred)
    if features==consecutive_features:

      print( f'classify on the given sentence using {features.__name__} features : ' )
      #print(consecutive_classify(sent_tokens))
    else:
      print( f'classify on the given sentence using {features.__name__} features : ' )
      #print(classify(features,classifier,vectorizer))

    comparison_table['question_step_number'].append(question_step_numbe)
    comparison_table['model_name'].append(model_name)
    comparison_table['features'].append(f'{features.__name__}')
    comparison_table['Training time'].append(Training_time)
    comparison_table['accuracy'].append(accuracy)


In [73]:
#x,y=crf_transform_to_dataset(pos_tagged_train_sentences,iob_tagged_train_sentences,features_list[0])

In [74]:
features_list=[consecutive_features,features,features_prev_next,features_bigram,features_prev_next_pos , features_word_pos]

In [75]:
for i in features_list:
  crf = sklearn_crfsuite.CRF(
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
  ) # c1 and c2 are regularization parameters
  vectorizer = DictVectorizer(sparse=False)
  crf_train(i,crf,vectorizer,'CRF','3.4',cutoff_size=5000)

classify on the given sentence using consecutive_features features : 
classify on the given sentence using features features : 
classify on the given sentence using features_prev_next features : 
classify on the given sentence using features_bigram features : 
classify on the given sentence using features_prev_next_pos features : 
classify on the given sentence using features_word_pos features : 


## [3.5]

In [76]:
tag_set =list(set([tag for sent in iob_tagged_train_sentences for (word,tag) in sent ]))
symbols =list(set([word for sent in iob_tagged_train_sentences for (word,tag) in sent ]))

print("Vocabulary lenght: " + str(len(symbols)))
print("Tagset lenght: " + str(len(tag_set)))

Vocabulary lenght: 4442
Tagset lenght: 3


In [77]:
# demonstrates IOB tagging using supervised training
from nltk.tag.hmm import HiddenMarkovModelTrainer

print('Training HMM...')
trainer = HiddenMarkovModelTrainer(tag_set, symbols)
hmm = trainer.train_supervised(iob_tagged_train_sentences,
    estimator=lambda fd, bins: nltk.probability.LidstoneProbDist(fd, 0.1, bins))

print('Testing...')
#hmm.test(labelled_sequences[:10], verbose=True)
print("Accurey is " + str(hmm.accuracy(iob_tagged_test_sentences)))

Training HMM...
Testing...
Accurey is 0.9734442489201728


In [78]:
    comparison_table['question_step_number'].append(3.5)
    comparison_table['model_name'].append("HMM")
    comparison_table['features'].append("none")
    comparison_table['Training time'].append('none')
    comparison_table['accuracy'].append(str(hmm.accuracy(iob_tagged_test_sentences)))

# Question [4]: DL Classification

## [4.1]


In [79]:
tagged_sentences=lines_to_lists(Data_df)

separate the words from the tags

In [80]:
import numpy as np

sentences, sentence_tags =[], []
for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(np.array(sentence))
    sentence_tags.append(np.array(tags))

# Let's see how a sequence looks

print(sentences[5])
print(sentence_tags[5])

['مقدمة' 'ابيضاض' 'الدم' '(' 'أو' 'اللوكيميا' ')' 'هو' 'اسم' 'يطلق' 'على'
 'مجموعة' 'من' 'سرطانات' 'خلايا' 'الدم' '.']
['O' 'B' 'I' 'O' 'O' 'B' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'B' 'I' 'I' 'O']


split the data

In [81]:
from sklearn.model_selection import train_test_split


(train_sentences,
 test_sentences,
 train_tags,
 test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2)

assigning to each word (and tag) a unique integer.
We’ll also add a special value for padding the sequences, and another one for unknown words .

In [82]:
words, tags = set([]), set([])

for s in train_sentences:
    for w in s:
        words.add(w.lower())

for ts in train_tags:
    for t in ts:
        tags.add(t)

word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs

tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to padding

converting the word dataset to integer dataset, both the words and the tags.

In [83]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []

for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    train_sentences_X.append(s_int)

for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    test_sentences_X.append(s_int)

for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])

for s in test_tags:
    test_tags_y.append([tag2index[t] for t in s])

print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])


[73, 2057, 2390, 1936, 4130, 3516, 2056, 744, 1936, 133, 921, 73, 1497, 944, 671, 725, 73, 1805, 253, 727, 539, 3171, 3643, 19, 2597, 2716, 1486, 472]
[1, 472]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1]


Dealing with Padding

In [84]:
MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH)

128


In [90]:
from keras.utils import pad_sequences

train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')

print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])


[  73 2057 2390 1936 4130 3516 2056  744 1936  133  921   73 1497  944
  671  725   73 1805  253  727  539 3171 3643   19 2597 2716 1486  472
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[  1 472   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

transform the sequences of tags to sequences of One-Hot Encoded tags. This is what the Dense Layer outputs

In [91]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [92]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print(cat_train_tags_y[0])

[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0.

For most of the sentences, the largest part is “padding tokens”. These are really easy to guess, hence the super high performance. Let’s write a custom accuracy, that ignores the paddings:

In [93]:
from keras import backend as K

def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)

        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

## [4.2]

In [94]:
from keras.models import Sequential
from keras.layers import InputLayer, Embedding, Bidirectional, LSTM, TimeDistributed, Dense, Activation
from keras.optimizers import Adam



In [95]:
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', ignore_class_accuracy(0)])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 128, 128)          566656    
                                                                 
 bidirectional (Bidirectiona  (None, 128, 512)         788480    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 128, 4)           2052      
 ibuted)                                                         
                                                                 
 activation (Activation)     (None, 128, 4)            0         
                                                                 
Total params: 1,357,188
Trainable params: 1,357,188
Non-trainable params: 0
_________________________________________________________________


In [96]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=40, validation_split=0.2)


Epoch 1/40
19/19 [==============================] - 54s 3s/step - loss: 0.4427 - accuracy: 0.8697 - ignore_accuracy: 0.4414 - val_loss: 0.1252 - val_accuracy: 0.9804 - val_ignore_accuracy: 0.8828
Epoch 2/40
19/19 [==============================] - 49s 2s/step - loss: 0.0793 - accuracy: 0.9815 - ignore_accuracy: 0.8771 - val_loss: 0.0574 - val_accuracy: 0.9857 - val_ignore_accuracy: 0.8972
Epoch 3/40
19/19 [==============================] - 45s 2s/step - loss: 0.0492 - accuracy: 0.9886 - ignore_accuracy: 0.9195 - val_loss: 0.0406 - val_accuracy: 0.9911 - val_ignore_accuracy: 0.9359
Epoch 4/40
19/19 [==============================] - 45s 2s/step - loss: 0.0384 - accuracy: 0.9918 - ignore_accuracy: 0.9411 - val_loss: 0.0352 - val_accuracy: 0.9929 - val_ignore_accuracy: 0.9501
Epoch 5/40
19/19 [==============================] - 53s 3s/step - loss: 0.0344 - accuracy: 0.9931 - ignore_accuracy: 0.9489 - val_loss: 0.1171 - val_accuracy: 0.9571 - val_ignore_accuracy: 0.7450
Epoch 6/40
12/19 [==

KeyboardInterrupt: ignored

In [97]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print(f"{model.metrics_names[1]}: {scores[1] * 100}")

24/24 [==============================] - 6s 257ms/step - loss: 0.0337 - accuracy: 0.9931 - ignore_accuracy: 0.9497
accuracy: 99.31108951568604


In [98]:
raw_sent = "التصلب المتعدد مرض يحتمل أن يسبب إعاقة في الدماغ والحبل الشوكي (الجهاز العصبي المركزي). عند الإصابة بمرض التصلب المتعدد، يهاجم الجهاز المناعي غمد الحماية (المايلين) الذي يغطي الألياف العصبية، ويسبب مشكلات في الاتصال بين الدماغ وبقية الجسم. في النهاية، يمكن أن يسبب المرض تلفًا أو تدهورًا دائمًا في الألياف العصبية وقد يؤدي الى امراض الدم."
sent_tokens = tokenization.tokenize(raw_sent)

In [99]:
sent_tokens_X = []
for s in sent_tokens:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    sent_tokens_X.append(s_int)

sent_tokens_X = pad_sequences(sent_tokens_X, maxlen=MAX_LENGTH, padding='post')
print(sent_tokens_X)

[[ 540  744    1 ...    0    0    0]
 [ 540  744    1 ...    0    0    0]
 [   1    1    1 ...    0    0    0]
 ...
 [   1    1    1 ...    0    0    0]
 [ 540  744 4342 ...    0    0    0]
 [ 472    0    0 ...    0    0    0]]


In [100]:
predictions = model.predict(sent_tokens_X)
print(predictions, predictions.shape)

3/3 [==============================] - 2s 180ms/step
[[[5.04757836e-03 9.43121552e-01 2.32017506e-02 2.86291689e-02]
  [5.49457222e-03 9.49484766e-01 1.98728759e-02 2.51477957e-02]
  [1.11884838e-02 9.36511755e-01 2.28166170e-02 2.94832308e-02]
  ...
  [9.99999940e-01 6.74817313e-10 1.06165938e-08 9.15892162e-09]
  [9.99999940e-01 8.01091804e-10 1.57132209e-08 1.31523468e-08]
  [9.99999940e-01 1.07165432e-09 2.43739962e-08 1.98434158e-08]]

 [[4.86651668e-03 9.42551315e-01 2.35968344e-02 2.89853998e-02]
  [5.18909236e-03 9.49248850e-01 2.01752409e-02 2.53867172e-02]
  [1.03051607e-02 9.36877966e-01 2.31301785e-02 2.96866763e-02]
  ...
  [9.99999940e-01 6.74815981e-10 1.06165539e-08 9.15892162e-09]
  [9.99999940e-01 8.01091804e-10 1.57131907e-08 1.31523210e-08]
  [9.99999940e-01 1.07165221e-09 2.43739962e-08 1.98433785e-08]]

 [[3.65367867e-02 9.01017368e-01 2.68492121e-02 3.55966203e-02]
  [7.00696632e-02 8.63818884e-01 2.80557089e-02 3.80556434e-02]
  [1.41998261e-01 7.88393319e-01 2.

In [101]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])

        token_sequences.append(token_sequence)

    return token_sequences

In [102]:
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

[['O', 'O', 'O', 'O', 'O', 'O', 'O', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', 

In [103]:
    comparison_table['question_step_number'].append('4.2')
    comparison_table['model_name'].append("LSTM")
    comparison_table['features'].append("none")
    comparison_table['Training time'].append('none')
    comparison_table['accuracy'].append(str(99.31108951568604))

## [4.3]

In [104]:
#pip install scikit-optimize

In [105]:
"""pip uninstall tensorflow
pip install tensorflow==2.12.0"""

'pip uninstall tensorflow\npip install tensorflow==2.12.0'

In [106]:
from skopt import BayesSearchCV

In [107]:
"""from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Define a function to create the model with hyperparameters
def create_model(embedding_dim, lstm_units, dropout_rate, learning_rate):
    model = Sequential()
    model.add(InputLayer(input_shape=(MAX_LENGTH, )))
    model.add(Embedding(len(word2index), embedding_dim))
    model.add(Bidirectional(LSTM(lstm_units, return_sequences=True)))
    model.add(TimeDistributed(Dense(len(tag2index))))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Create a KerasClassifier based on the model creation function
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the hyperparameters to tune
param_grid = {
    'embedding_dim': [64, 128, 256],
    'lstm_units': [128, 256, 512],
    'dropout_rate': [0.2, 0.5],
    'learning_rate': [0.001, 0.01]
}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(train_sentences_X, to_categorical(train_tags,len(tag2index)))  # Provide your training data

# Print the best hyperparameters and corresponding accuracy
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))"""

'from sklearn.model_selection import GridSearchCV\nfrom tensorflow.keras.wrappers.scikit_learn import KerasClassifier\n\n# Define a function to create the model with hyperparameters\ndef create_model(embedding_dim, lstm_units, dropout_rate, learning_rate):\n    model = Sequential()\n    model.add(InputLayer(input_shape=(MAX_LENGTH, )))\n    model.add(Embedding(len(word2index), embedding_dim))\n    model.add(Bidirectional(LSTM(lstm_units, return_sequences=True)))\n    model.add(TimeDistributed(Dense(len(tag2index))))\n    model.add(Activation(\'softmax\'))\n    model.compile(loss=\'categorical_crossentropy\',\n                  optimizer=Adam(learning_rate),\n                  metrics=[\'accuracy\'])\n    return model\n\n# Create a KerasClassifier based on the model creation function\nmodel = KerasClassifier(build_fn=create_model, verbose=0)\n\n# Define the hyperparameters to tune\nparam_grid = {\n    \'embedding_dim\': [64, 128, 256],\n    \'lstm_units\': [128, 256, 512],\n    \'dropou

The Comparison Dictionary example

In [108]:
"""comparison_table = {}

comparison_table['question_step_number'] = []
comparison_table['model_name'] = []
comparison_table['features'] = []
comparison_table['accuracy'] = []"""

"comparison_table = {}\n\ncomparison_table['question_step_number'] = []\ncomparison_table['model_name'] = []\ncomparison_table['features'] = []\ncomparison_table['accuracy'] = []"

In [109]:
import pandas as pd

df = pd.DataFrame(comparison_table)
df

,question_step_number,model_name,features,Training time,accuracy
0,3.2,Multinomial naive bayes,consecutive_features,0:00:00.553024,0.979123
1,3.2,Multinomial naive bayes,features,0:00:00.195037,0.958887
2,3.2,Multinomial naive bayes,features_prev_next,0:00:00.160412,0.968725
3,3.2,Multinomial naive bayes,features_bigram,0:00:00.047619,0.961126
4,3.2,Multinomial naive bayes,features_prev_next_pos,0:00:00.052280,0.964166
5,3.2,Multinomial naive bayes,features_word_pos,0:00:00.066995,0.956967
6,3.3,Multinomial Logistic Regression,consecutive_features,0:00:09.114858,0.984802
7,3.3,Multinomial Logistic Regression,features,0:00:10.303652,0.971525
8,3.3,Multinomial Logistic Regression,features_prev_next,0:00:10.426066,0.981763
9,3.3,Multinomial Logistic Regression,features_bigram,0:00:05.495583,0.970965


<h1 style="text-align: center;">Sequence to Sequence<h1>

Prepare libraries

In [111]:
# here put every import you need e.g. import nltk
# it's better to load what you need from the package by from [] import [] instead of import the whole package

Download data

In [ ]:
"""!wget 'https://drive.google.com/uc?export=download&id=1OoUGXNKnmm3KYabreBFWMV9mr2JwQOJH' -O 'test.csv'
!wget 'https://drive.google.com/uc?export=download&id=1xnw6kKitwQwYgaOAQppDbj6T3ARCwRqz' -O 'train.csv'"""

# [Extra 1]

# [2]

# [3]